# Подгрузка 

In [1]:
!pip install transformers
!pip install pymorphy2
!pip install pymystem3
!pip install transformers[sentencepiece]
!pip install evaluate
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Датасет

In [2]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
df = pd.read_excel("/content/drive/MyDrive/1 плох раб.xlsx")
df['target'] = [1 if elem == 'Неисправность инфраструктуры/подвижного состава' \
                else 0 for elem in df['Причина возникновения общая']]
df = df[['Примечание', 'target']]
df = df.rename(columns={'Примечание': 'text'})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Трансформер + токенезация

In [16]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
import torch
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [38]:
MODEL_NAME = 'DmitryPogrebnoy/distilbert-base-russian-cased'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
NUM_EPOCHS = 1
SEED = 42

In [39]:
def f1_score(predictions, labels):
  tp = (labels * predictions).sum().float()
  fp = ((1 - labels) * predictions).sum().float()
  fn = (labels * (1 - predictions)).sum().float()
  precision = tp / (tp + fp + 1e-8)
  recall = tp / (tp + fn + 1e-8)
  f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)
  # print('F1 score:', f1_score.item())
  return f1_score

In [40]:
class MyDataset(Dataset):
  def __init__(self, data, labels):
    self.data = data
    self.labels = labels

  def __getitem__(self, index):
    return self.data[index], self.labels[index]

  def __len__(self):
    return len(self.data)

In [124]:
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME, 
                                                      num_labels=2,
                                                      output_hidden_states=True, 
                                                      output_attentions=True).to(DEVICE)

Some weights of the model checkpoint at DmitryPogrebnoy/distilbert-base-russian-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at DmitryPogrebnoy/distilbert-base-russian-cased and are newly initialized: ['pre_classi

In [125]:
df = df.iloc[:50]
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['target'].values, 
                                                    test_size=0.2, random_state=SEED)

train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset,  shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
sheduler = None



# num_training_steps = NUM_EPOCHS * len(train_dataloader)

# lr_scheduler = get_scheduler(
# 	name='linear',
# 	optimizer=optimizer,
# 	num_warmup_steps=0,
# 	num_training_steps=num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [126]:
metric = []
embeddings = []
# Обучение
for epoch in tqdm(range(NUM_EPOCHS)):
  model.train()
  for x_batch, y_batch in tqdm(train_dataloader):
      inputs = tokenizer.batch_encode_plus(x_batch, 
                                           add_special_tokens=True, 
                                           max_length=512,
                                           padding=True, 
                                           return_tensors='pt').to(DEVICE)
      input_ids = inputs['input_ids']
      attention_mask = inputs['attention_mask']
      labels = y_batch.clone().detach().to(DEVICE)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      #scheduler.step()
      optimizer.zero_grad()
  val_size = 0.
  val_score = 0.

  # Валидация
  model.eval()
  for x_batch, y_batch in tqdm(test_dataloader):
      inputs = tokenizer.batch_encode_plus(x_batch, 
                                           add_special_tokens=True, 
                                           max_length=512,
                                           padding=True, 
                                           return_tensors='pt').to(DEVICE)
      input_ids = inputs['input_ids']
      attention_mask = inputs['attention_mask']
      labels = y_batch.clone().detach().to(DEVICE)

      with torch.no_grad():
          outputs = model(input_ids, attention_mask=attention_mask)
      embeddings.append(outputs)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      val_score += f1_score(predictions, y_batch)
      val_size += 1
        
  metric.append(val_score / val_size)

  print(f'epoch {epoch} - {metric[-1]}')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 0 - 0.761904776096344


In [44]:
# Extract feature importance
tokens = list(tokenizer.get_vocab().keys())
weights = model.distilbert.embeddings.word_embeddings.weight.detach().cpu().numpy()
feature_importance = pd.DataFrame(index=tokens, data=weights)

In [52]:
feature_importance[feature_importance.iloc[:, 10] > 0.1]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
Ъ,-0.047918,0.004191,-0.054596,0.008829,-0.102717,-0.068353,0.013024,-0.062458,-0.086087,0.032645,...,0.014725,-0.044838,-0.059317,0.149024,0.037577,0.032429,0.056825,-0.023788,0.134397,-0.087350
об,-0.036798,0.015736,0.029083,-0.029689,-0.066384,-0.023766,0.017054,-0.034697,-0.071565,0.048049,...,-0.044204,-0.039268,0.011991,-0.041847,-0.010349,0.044879,-0.000580,0.016784,-0.026533,-0.013691
##ман,-0.026742,0.010768,-0.039600,-0.036229,-0.012212,-0.019571,-0.002768,-0.021733,-0.033330,0.033206,...,0.019695,-0.060054,0.005417,0.005840,-0.033723,-0.058102,-0.015009,-0.058030,0.020956,-0.012629
##ую,0.009465,-0.018722,0.010573,-0.036490,-0.068642,-0.030937,-0.006782,-0.004530,-0.009793,0.038867,...,0.047326,-0.025802,-0.052832,-0.023014,-0.004962,0.014653,-0.022036,-0.032295,0.042208,-0.020748
ад,0.029249,0.014258,0.104398,-0.062689,-0.033849,0.057614,0.015825,0.083883,-0.038490,0.058387,...,-0.081803,-0.017366,-0.030079,-0.065564,0.018015,-0.075396,0.011182,-0.026723,-0.027188,0.052775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
##иже,-0.050564,-0.004207,-0.023110,0.074248,-0.060641,0.003937,0.033429,0.099686,-0.046468,0.063874,...,0.046557,-0.024759,-0.021639,-0.025077,-0.023806,-0.016250,-0.075699,-0.028765,-0.061925,0.020136
фотографска,-0.074406,-0.046301,-0.065639,0.058766,0.033496,0.046016,0.033331,0.005712,-0.056607,0.006627,...,0.059095,0.035348,0.003978,0.001037,-0.000738,-0.033335,-0.007465,0.054132,-0.044225,0.023539
##ционная,0.011497,-0.018294,-0.016461,0.043168,-0.043452,-0.022970,0.067938,-0.046279,-0.067401,0.003012,...,-0.063034,0.042498,0.016987,-0.026554,0.005528,-0.009018,0.003675,-0.127507,-0.011628,-0.050947
кафе,0.074161,0.001027,0.065421,-0.027195,-0.011321,0.022921,0.001622,-0.035036,-0.078328,-0.068463,...,-0.036799,0.021461,-0.007211,-0.026640,0.055958,-0.100896,0.000272,-0.001788,0.011746,-0.034343


In [127]:
text ='В салоне трамвая стало плохо женщине'
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.encode(text, return_tensors='pt')

with torch.no_grad():
    outputs = model(input_ids)
    hidden_states = outputs[2]

last_layer = hidden_states[-1].squeeze(0)
importance = torch.sum(last_layer, dim=0).detach().numpy()

important_indexes = importance.argsort()[-2:]
important_tokens = [tokens[i-2] for i in important_indexes[0]]

len(important_tokens)

15

In [97]:
attention_scores = outputs.attentions[-1][0]
print(attention_scores[4].sum(dim=0))

# for i, word in enumerate(text.split()):
#     print(word, attention_scores[i:i+1].sum())

tensor([1.1838, 0.4786, 0.8640, 0.8690, 0.2821, 0.7540, 0.8738, 0.9088, 1.6142,
        0.7507, 0.5453, 0.7274, 2.8328, 1.1509, 1.1647])


In [100]:
attention = outputs.attentions  # attention weights for each layer

# Obtain average attention weights across all layers
attention_weights = torch.stack(attention, dim=0).mean(dim=0)

# Compute word importance scores
word_importance = attention_weights.sum(dim=1) / attention_weights.shape[1]
print(word_importance)

tensor([[[0.3724, 0.0407, 0.0202, 0.0242, 0.0187, 0.0177, 0.0202, 0.0222,
          0.0673, 0.0160, 0.0143, 0.0175, 0.0595, 0.0691, 0.2199],
         [0.2984, 0.0905, 0.0864, 0.0411, 0.0563, 0.0186, 0.0161, 0.0213,
          0.0739, 0.0133, 0.0138, 0.0170, 0.0411, 0.0364, 0.1759],
         [0.2226, 0.0924, 0.1055, 0.1555, 0.0398, 0.0226, 0.0203, 0.0308,
          0.0350, 0.0099, 0.0113, 0.0138, 0.0400, 0.0165, 0.1840],
         [0.2389, 0.0549, 0.1331, 0.0964, 0.0870, 0.0334, 0.0244, 0.0267,
          0.0294, 0.0107, 0.0094, 0.0158, 0.0246, 0.0164, 0.1989],
         [0.2104, 0.0875, 0.0498, 0.1194, 0.0827, 0.0499, 0.0212, 0.0439,
          0.0774, 0.0116, 0.0097, 0.0132, 0.0426, 0.0381, 0.1427],
         [0.2308, 0.0391, 0.0281, 0.0455, 0.0347, 0.0711, 0.1333, 0.0855,
          0.0575, 0.0172, 0.0135, 0.0180, 0.0363, 0.0233, 0.1661],
         [0.2454, 0.0264, 0.0244, 0.0310, 0.0248, 0.1050, 0.1001, 0.1260,
          0.0481, 0.0157, 0.0120, 0.0134, 0.0265, 0.0213, 0.1801],
         [0.2

In [ ]:
!pip install bertviz

In [133]:
attention = outputs.attentions[-1][0]
attention = torch.stack(attention, dim=0).mean(dim=0)
print(attention.shape)
# Visualize the attention matrix
model_view(attention, input_ids)

TypeError: ignored

In [156]:
len(tokens)

13

In [155]:
attention_scores = outputs.hidden_states[-1].sum(dim=0)
print(attention_scores.shape)
# Get indices of words sorted by attention score
sorted_indices = attention_scores.argsort(descending=True)

# Print top 5 most important words in the sentence
for i in range(5):
    index = sorted_indices[i][2]
    print(index)
    word = tokens[index]
    score = attention_scores[index].item()
    print(f"{i + 1}. '{word}' (attention score: {score:.2f})")

torch.Size([15, 768])
tensor(131)


IndexError: ignored

In [143]:
# get contextualized word embeddings for each token
embeddings = outputs.hidden_states
print(len(embeddings))
# compute importance of each bigram using pointwise mutual information
pmi_scores = []
for i in range(len(embeddings)-1):
    bigram_embedding = np.concatenate([embeddings[i], embeddings[i+1]])
    pmi_score = compute_pmi(bigram_embedding)
    print(pmi_score)
    pmi_scores.append(pmi_score)

# find the 10 bigrams with highest importance
indices = np.argsort(pmi_scores)[-10:]
important_bigrams = []
for index in indices:
    important_bigram = tokens[index] + " " + tokens[index+1]
    important_bigrams.append(important_bigram)

7


NameError: ignored

In [138]:
important_bigrams

[]

In [ ]:
input_ids_ = []
attention_masks_ = []
for text in df['text']:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 512,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt', truncation=True).to(DEVICE)
    
    input_ids_.append(encoded_dict['input_ids'])
    attention_masks_.append(encoded_dict['attention_mask'])
    
    input_ids = torch.cat(input_ids_, dim=0).to(DEVICE)
    attention_masks = torch.cat(attention_masks_, dim=0).to(DEVICE)
    outputs = model(input_ids=input_ids, attention_mask=attention_masks)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

    hidden_states = outputs.hidden_states
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=-1)
    token_embeddings = token_embeddings.permute(1,0,2)
    pooled = outputs.pooler_output
    
    df_bert = pd.DataFrame()
    for i in range(token_embeddings.shape[0]):
        layer_embeddings = token_embeddings[i]
        layer_embeddings = layer_embeddings.detach().numpy()
        for j in range(layer_embeddings.shape[0]):
            col_name = f"layer_{i}_feature_{j}"
            df_bert[col_name] = layer_embeddings[j]
    df_pooled = pd.DataFrame(pooled.detach().numpy())
    df_bert = pd.concat([df_bert, df_pooled], axis=1)
    
    return df_bert

# Example usage:


bert_features = get_bert_features(df.iloc[:100])


In [4]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example input text
text = "This is a sample input for BERT trigram extraction."

# Tokenize input text and obtain token IDs and attention mask
input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
attention_mask = torch.tensor([[1] * len(input_ids[0])])

# Apply BERT model to obtain embeddings
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    embeddings = outputs.last_hidden_state

# Calculate trigram weights by summing up corresponding embedding vectors
trigram_weights = []
for i in range(len(embeddings[0]) - 2):
    trigram_weights.append(torch.sum(embeddings[0][i:i+3], dim=0))

 # Extract top trigrams by weight
top_trigrams = torch.topk(torch.stack(trigram_weights), k=5).indices

# Convert top trigrams back to text
output = []
for trigram in top_trigrams:
    output.append(tokenizer.convert_ids_to_tokens(trigram + 1))
final_output = [t for t in output if '[PAD]' not in t and '[CLS]' not in t and '[SEP]' not in t]
print(final_output)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[['[unused81]', '[unused564]', '[unused201]', '[unused567]', '[unused584]'], ['[unused564]', '[unused81]', '[unused567]', '[unused221]', '[unused719]'], ['[unused221]', '[unused567]', '[unused128]', '[unused138]', '[unused238]'], ['[unused221]', '[unused138]', '[unused567]', '[unused400]', '[unused653]'], ['[unused221]', '[unused182]', '[unused138]', '[unused553]', '[unused207]'], ['[unused221]', '[unused553]', '[unused237]', '[unused138]', '[unused182]'], ['[unused221]', '[unused182]', '[unused553]', '[unused479]', '[unused563]'], ['[unused221]', '[unused182]', '[unused177]', '[unused323]', '[unused15]'], ['[unused221]', '[unused182]', '[unused177]', '[unused15]', '[unused103]'], ['[unused119]', '[unused635]', '[unused103]', '[unused458]', '[unused702]'], ['[unused119]', '[unused635]', '[unused224]', '[unused269]', '[unused479]']]


In [6]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "The quick brown fox jumps over the lazy dog"

# Tokenize the text and get the trigrams
tokens = tokenizer.tokenize(text)
trigrams = [tokens[i:i+3] for i in range(len(tokens)-2)]

# Print the trigrams
print(trigrams)

[['the', 'quick', 'brown'], ['quick', 'brown', 'fox'], ['brown', 'fox', 'jumps'], ['fox', 'jumps', 'over'], ['jumps', 'over', 'the'], ['over', 'the', 'lazy'], ['the', 'lazy', 'dog']]


In [7]:
from transformers import BertTokenizer
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "Your input text here"
max_length = 128

# Tokenize the text and truncate if needed
tokenized_text = tokenizer.encode(text, add_special_tokens=True, max_length=max_length)

# Loop over the tokens and extract trigrams
trigrams = []
for i in range(0,len(tokenized_text)-2):
    trigrams.append(tokenizer.decode(tokenized_text[i:i+3]))

# Get the attention scores for the input text
attention_scores = np.ones((1, len(tokenized_text)), dtype=np.float32)

# Compute the importance score for each trigram
trigram_scores = np.zeros(len(trigrams), dtype=np.float32)
for i in range(len(trigrams)):
    trigram_token_ids = tokenizer.encode(trigrams[i])
    trigram_attention_scores = np.ones((1, len(trigram_token_ids)), dtype=np.float32)
    trigram_scores[i] = np.sum(trigram_attention_scores * attention_scores[:, i:i+len(trigram_token_ids)])
    
# Print the top 10 most important trigrams
top_indices = np.argsort(trigram_scores)[::-1][:10]
for index in top_indices:
    print(trigrams[index], trigram_scores[index])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: ignored

In [23]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "sample text to extract trigrams from"

trigrams = set()

# Tokenize the text and get all the trigrams
for token in tokenizer.tokenize(text):
  if len(token) >= 3:
    for i in range(len(token) - 2):
      trigrams.add(token[i:i+3])

# Sort the trigrams by their frequency
trigrams_freq = {trigram: 0 for trigram in trigrams}
for token in tokenizer.tokenize(text):
  if len(token) >= 3:
    for i in range(len(token) - 2):
      trigram = token[i:i+3]
      trigrams_freq[trigram] += 1

trigrams_freq_sorted = sorted(trigrams_freq.items(), key=lambda x:x[1], reverse=True)

# Print the top 10 trigrams by frequency
for trigram, freq in trigrams_freq_sorted[:10]:
  print(trigram, freq)


ext 2
mpl 1
##s 1
rom 1
sam 1
xtr 1
tra 1
##g 1
act 1
gra 1


In [29]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Fine-tune BERT on your specific task/dataset
# ...

# Extract trigrams from top k most important embeddings
input_text = "your text here"
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_tensors='tf')
outputs = bert_model(input_ids)
embeddings = outputs.last_hidden_state

# Aggregate trigrams from top k most important embeddings
k = 5
top_k_indices = tf.math.top_k(tf.linalg.norm(embeddings, axis=-1), k=k).indices
trigrams = []
for i in top_k_indices:
  token_ids = input_ids[0][i:i+3]
  trigram = tokenizer.decode(token_ids)
  trigrams.append(trigram)

print(trigrams)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


TypeError: ignored

In [30]:
import transformers
import numpy as np

# Load the BERT model
model_name = 'bert-base-cased'
model = transformers.AutoModel.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# Get the input text and tokenize it
input_text = "This is a sample input text."
input_tokens = tokenizer.tokenize(input_text)

# Convert the tokens to input IDs and create an input tensor
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_tensor = torch.tensor([input_ids])

# Get the BERT outputs
with torch.no_grad():
    outputs = model(input_tensor)

# Get the embedding for the first token
embedding = outputs[0][0][0]

# Extract the trigrams from the embedding
trigram_indices = np.arange(0, len(input_tokens) - 2)
trigrams = [embedding[i:i+3] for i in trigram_indices]


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.cluster import KMeans

# Load the BERT model and the tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fine-tune the model on your specific dataset
train_dataset = MyDataset(...)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=32)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
model.train()

for epoch in range(3):
    for batch in train_dataloader:
        inputs = tokenizer.batch_encode_plus(batch, add_special_tokens=True, max_length=512, padding=True, return_tensors='pt')
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        labels = torch.tensor(batch.labels)
        loss, outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss.backward()
        optimizer.step()
        model.zero_grad()

# Use the model to generate embeddings
test_dataset = MyDataset(...)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)
model.eval()

embeddings = []
for batch in test_dataloader:
    inputs = tokenizer.batch_encode_plus(batch, add_special_tokens=True, max_length=512, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    with torch.no_grad():
        _, outputs = model(input_ids, attention_mask=attention_mask)
    embeddings.append(outputs)

# Use clustering or dimensionality reduction to identify important trigrams
kmeans = KMeans(n_clusters=10, random_state=0).fit(embeddings)
important_trigrams = kmeans.cluster_centers_

# Then you can use the important trigrams for further analysis or classification tasks


[tensor([0.0395, 0.3526, 0.1671]),
 tensor([ 0.3526,  0.1671, -0.2175]),
 tensor([ 0.1671, -0.2175, -0.5806]),
 tensor([-0.2175, -0.5806, -0.2186]),
 tensor([-0.5806, -0.2186,  0.6767])]

In [32]:
import torch
from transformers import BertModel, BertTokenizer
from itertools import combinations
from scipy.spatial.distance import cosine

# Load pre-trained BERT model and tokenizer
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define input text
text = "This is a test sentence for BERT transformers"

# Tokenize input text
tokens = tokenizer.tokenize(text)

# Obtain contextualized embeddings for each token
with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)[0][0]
    embeddings = outputs.detach().numpy()

# Extract all trigrams from the tokenized text
trigrams = list(combinations(tokens, 3))

# Find the trigram with the highest cosine similarity score
max_score = -1
max_trigram = None
for trigram in trigrams:
    vec1 = embeddings[tokenizer.convert_tokens_to_ids([trigram[0]])[0]]
    vec2 = embeddings[tokenizer.convert_tokens_to_ids([trigram[1]])[0]]
    vec3 = embeddings[tokenizer.convert_tokens_to_ids([trigram[2]])[0]]
    score = 1 - cosine(vec1 + vec2 + vec3, embeddings.sum(axis=0))
    if score > max_score:
        max_score = score
        max_trigram = trigram

# Print the trigram with the highest cosine similarity score
print(max_trigram)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


IndexError: ignored

In [33]:
from transformers import BertTokenizer

text = "Here is some sample text to tokenize and extract important trigrams from."

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize(text)

trigrams = []
for i in range(len(tokens)-2):
  trigrams.append((tokens[i], tokens[i+1], tokens[i+2]))

important_trigrams = []
for trigram in trigrams:
  score = calculate_importance(trigram)  # replace with your own function to calculate importance score
  if score > threshold:  # replace with your own threshold
    important_trigrams.append(trigram)

NameError: ignored

In [ ]:
trigrams = []
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
for i in range(0, len(tokens)-2):
  trigrams.append((tokens[i], tokens[i+1], tokens[i+2]))

important_trigrams = []
for i in range(0, len(last_hidden_state[0])):
  if (i+2 < len(last_hidden_state[0])):
    trigram_embedding = torch.cat((last_hidden_state[0][i], last_hidden_state[0][i+1], last_hidden_state[0][i+2]), 0)
    important_trigrams.append((trigrams[i], trigram_embedding))


In [35]:
# Load BERT model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", output_attentions=True)

# Example text input
text = "This is an example sentence for BERT."

# Tokenize input
input_ids = tokenizer.encode(text, return_tensors='pt')

# Get attention weights from BERT model
outputs = model(input_ids)
attention_weights = outputs.attentions.squeeze().mean(0)

# Identify important trigrams
importances = attention_weights.sum(0)
trigram_importances = [(tokenizer.decode(trigram), importances[trigram[0], trigram[1], trigram[2]].item()) for idx, trigram in enumerate(zip(*[iter(input_ids[0])] * 3))]
trigram_importances_sorted = sorted(trigram_importances, key=lambda x: x[1], reverse=True)
top_trigrams = trigram_importances_sorted[:10]

# Print top trigrams
for trigram, importance in top_trigrams:
    print(trigram, importance)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

AttributeError: ignored

In [ ]:
import torch

trigrams = []
output_tokens = output.last_hidden_state[0]
for i in range(len(output_tokens) - 2):
    trigram = [tokenizer.decode([output_tokens[i+j].argmax().item()]) for j in range(3)]
    trigrams.append(tuple(trigram))